In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        
    def __str__(self):
        return 'surface: {0}, base: {1}, pos: {2}, po1: {3}'.format(self.surface, self.base, self.pos, self.pos1)

In [2]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []
        self.surfaces2 = ''
        
        for morph in self.morphs:
            if morph.pos != '記号':
                self.surfaces2 += morph.surface
        
    def __str__(self):
        return 'surfaces2: {0}, dst: {1}, srcs: {2}'.format(self.surfaces2, self.dst, self.srcs)

In [3]:
def make_edges(sentence):
    import CaboCha
    
    Chunks = []
    Chunk_Morphs = []
    EOS_count = 0
    src_dst = []
    edges = []
    
    c = CaboCha.Parser('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    c_sentence = c.parse(sentence).toString(CaboCha.FORMAT_LATTICE)
    lines = c_sentence.split('\n')
    for line in lines:
        if line[0] == '*':
            if len(Chunk_Morphs) > 0:
                Chunks.append(Chunk(Chunk_Morphs, phrase_info[2][:-1]))
            Chunk_Morphs = []
            phrase_info = line.split(' ')            
            src_dst.append([phrase_info[1], phrase_info[2][:-1]])
            continue
        if line == 'EOS':
            if len(Chunk_Morphs) > 0:
                Chunks.append(Chunk(Chunk_Morphs, phrase_info[2][:-1]))
            for s_d in src_dst:
                if int(s_d[1]) != -1:
                    Chunks[int(s_d[1])].srcs.append(s_d[0])
            break
        morpheme = line.split('\t')
        surface = morpheme[0]
        base = morpheme[1].split(',')[6]
        pos = morpheme[1].split(',')[0]
        pos1 = morpheme[1].split(',')[1]
        Chunk_Morphs.append(Morph(surface, base, pos, pos1))
        
    for i, c in enumerate(Chunks):                
        if c.dst != '-1':
            edges.append([i, c.surfaces2, c.dst, Chunks[int(c.dst)].surfaces2])
            
    return edges

In [4]:
def make_graph(edges):
    import pydot_ng as pydot
    
    graph = pydot.Dot(graph_type = 'digraph')
    for edge in edges:
        graph.add_node(pydot.Node(edge[0], label = edge[1]))
        graph.add_node(pydot.Node(edge[2], label = edge[3]))
        graph.add_edge(pydot.Edge(edge[0], edge[2]))
        
    return graph

In [5]:
sentence = input('enter sentence: ')
edges = make_edges(sentence)
if len(edges) > 0:
    graph = make_graph(edges)
    graph.write_png('results/44_result.png')

enter sentence: 吾輩はここで始めて人間というものを見た。
